In [ ]:
# !Convert*overall_image.py*w*sehx*

In [ ]:
# !Switch*
from handler import FigureHandler
from dataset import ot_im_dotmark
from stats import Statistics
# !SwitchCase*
# import font
# from handler import FigureHandler
# from dataset import ot_im_dotmark
# from stats import Statistics
# !SwitchEnd*

In [ ]:
from solver_mosek import solve_mosek_primal_simplex, solve_mosek_dual_simplex, solve_mosek_interior_point
from solver_gurobi import solve_gurobi_primal_simplex, solve_gurobi_dual_simplex, solve_gurobi_barrier
from first_ADMM_primal import solve_ADMM_primal
from first_grad_primal import solve_grad_primal, solve_combine
from discrete_transportation_simplex import solve_transportation_simplex
from multiscale_image_mosek import solve_multiscale_image_mosek_primal_simplex

In [ ]:
# !Switch*
fh = FigureHandler(redir=True)
# !SwitchCase*
# fh = FigureHandler(sav=True, disp=False, ext=".pgf", redir=True)
# !SwitchEnd*

In [ ]:
cls_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
rep = 1

In [ ]:
def suite(stat):
    stat.set_objx(solve_mosek_interior_point, prog=fh.write)
    stat.test(solve_mosek_primal_simplex, prog=fh.write)
    stat.output_last()
    stat.test(solve_mosek_dual_simplex, prog=fh.write)
    stat.output_last()
    stat.test(solve_mosek_interior_point, prog=fh.write)
    stat.output_last()
    stat.test(solve_gurobi_primal_simplex, prog=fh.write)
    stat.output_last()
    stat.test(solve_gurobi_dual_simplex, prog=fh.write)
    stat.output_last()
    stat.test(solve_gurobi_barrier, prog=fh.write)
    stat.output_last()
    stat.test(
        solve_combine,
        cfgs=[
            {
                "func": solve_ADMM_primal,
                "args": (),
                "kwargs": dict(
                    its=[5000],
                    rhos=[3.],
                    alphas=[1.618],
                    epss=[1e-3],
                )
            },
            {
                "func": solve_grad_primal,
                "args": (),
                "kwargs": dict(
                    its=[800, 800, 800],
                    pi0s=[10., 10., 10.],
                    pi1s=[1000., 10000., 100000.],
                    pi2s=[1000., 10000., 100000.],
                    lrs=[1e-6, 1e-7, 1e-8],
                )
            },
        ],
        prog=fh.write,
    )
    stat.output_last()
    stat.test(
        solve_transportation_simplex,
        eps=1e-12,
        it=20000,
        prog=fh.write,
    )
    stat.output_last()
    stat.test(
        solve_multiscale_image_mosek_primal_simplex,
        stop=4, shrink=2,
        caps=[1., 0.1, 0.1, 0.1],
        errs=[0.001, 0.001, 0.001, 0.00],
        prog=fh.write,
    )
    stat.output_last()

In [ ]:
for cls in cls_list:
    stat = Statistics(
        probs=[
            ot_im_dotmark(cls, 32, i, log=fh.write) for i in range(1, rep+1)
        ],
        prob="dotmark class {0} 32*32".format(cls),
        log=fh.write,
    )
    suite(stat)

In [ ]:
# !ConvertEnd*